## test netcdf+

This is a more extensive integration test, if all the features of netcdf+ work as expected.

In [1]:
import openpathsampling as paths
from openpathsampling.netcdfplus import NetCDFPlus, ObjectStore, StorableObject
import numpy as np

In [2]:
class Node(StorableObject):
    def __init__(self, value):
        self.value = value
        
    def __repr__(self):
        return 'Node(%s)' % self.value

### 1. Open new storage

try to create a new storage

In [3]:
st = NetCDFPlus('test_netcdfplus.nc', mode='w')

### Create some content

In [4]:
st.create_store('nodes', ObjectStore(Node, has_name=False))

Initialize the store

In [5]:
st.finalize_stores()

In [6]:
st.nodes.save(Node(10));

In [7]:
st.close()

### 2. Reopen empty storage

In [8]:
st = NetCDFPlus('test_netcdfplus.nc', mode='a')

set caching of the new store

In [9]:
st.nodes.set_caching(10)

Check if the stores were correctly loaded

In [10]:
assert('nodes' in st.objects)

In [11]:
assert('stores' in st.objects)

In [12]:
assert(len(st.nodes) == 1)

In [13]:
assert(len(st.stores) == 1)

### create variables types

Get a list of all possible variable types

In [14]:
print sorted(st.get_var_types())

['bool', 'float', 'index', 'int', 'json', u'lazyobj.nodes', 'lazyobj.stores', 'length', 'long', 'numpy.float32', 'numpy.float64', 'numpy.int16', 'numpy.int32', 'numpy.int64', 'numpy.int8', 'numpy.uint16', 'numpy.uint32', 'numpy.uint64', 'numpy.uint8', u'obj.nodes', 'obj.stores', 'store', 'str']


Make a dimension on length 2 to simplify dimension nameing.

Now we construct for each type a corresponding variable of dimensions 2x2x2.

In [15]:
st.create_dimension('pair', 2)

In [16]:
for var_type in st.get_var_types():
    st.create_variable(var_type, var_type, dimensions=('pair', 'pair', 'pair'))

In [17]:
st.update_delegates()

In [18]:
for var_name, var in sorted(st.variables.items()):
    print var_name, var.dimensions

bool (u'pair', u'pair', u'pair')
float (u'pair', u'pair', u'pair')
index (u'pair', u'pair', u'pair')
int (u'pair', u'pair', u'pair')
json (u'pair', u'pair', u'pair')
lazyobj.nodes (u'pair', u'pair', u'pair')
lazyobj.stores (u'pair', u'pair', u'pair')
length (u'pair', u'pair', u'pair')
long (u'pair', u'pair', u'pair')
nodes_json (u'nodes',)
numpy.float32 (u'pair', u'pair', u'pair')
numpy.float64 (u'pair', u'pair', u'pair')
numpy.int16 (u'pair', u'pair', u'pair')
numpy.int32 (u'pair', u'pair', u'pair')
numpy.int64 (u'pair', u'pair', u'pair')
numpy.int8 (u'pair', u'pair', u'pair')
numpy.uint16 (u'pair', u'pair', u'pair')
numpy.uint32 (u'pair', u'pair', u'pair')
numpy.uint64 (u'pair', u'pair', u'pair')
numpy.uint8 (u'pair', u'pair', u'pair')
obj.nodes (u'pair', u'pair', u'pair')
obj.stores (u'pair', u'pair', u'pair')
store (u'pair', u'pair', u'pair')
stores_json (u'stores',)
stores_name (u'stores',)
str (u'pair', u'pair', u'pair')


In [19]:
for var in sorted(st.vars):
    print var

bool
float
index
int
json
lazyobj.nodes
lazyobj.stores
length
long
nodes_json
numpy.float32
numpy.float64
numpy.int16
numpy.int32
numpy.int64
numpy.int8
numpy.uint16
numpy.uint32
numpy.uint64
numpy.uint8
obj.nodes
obj.stores
store
stores_json
stores_name
str


#### Bool

In [20]:
st.vars['bool'][:] = True

In [21]:
print st.vars['bool'][:]

[[[True, True], [True, True]], [[True, True], [True, True]]]


#### Float

In [22]:
st.vars['float'][1,1] = 1.0

In [23]:
print st.vars['float'][:]

[[[None, None], [None, None]], [[None, None], [1.0, 1.0]]]


#### Index
`Index` is special in the sense that it supports only integers that are non-negative. Negative values will be interpreted as `None`

In [24]:
st.vars['index'][0,1,0] = 10
st.vars['index'][0,1,1] = -1
st.vars['index'][0,0] = None

In [25]:
print st.vars['index'][0,1]
print st.vars['index'][0,0]

[10, None]
[None, None]


#### Int

In [26]:
st.vars['int'][0,1,0] = 10
st.vars['int'][0,1,1] = -1

In [27]:
print st.vars['int'][:]

[[[None, None], [10, -1]], [[None, None], [None, None]]]


#### JSON

A JSON serializable object. This can be normal very simple python objects, plus numpy arrays, and objects that implement `to_dict` and `from_dict`.

In [28]:
st.vars['json'][0,1,1] = {'Hallo': 2, 'Test': 3}

In [29]:
st.vars['json'][0,1,0]

In [30]:
st.vars['json'][0,1,0] = Node(10)

In [31]:
print st.variables['json'][0,1,:]

[u'{"_cls": "Node", "_dict": {"value": 10}}' u'{"Test": 3, "Hallo": 2}']


All object types registered as being Storable by subclassing from `openpathsampling.base.StorableObject`. For this test we only register type `paths.Snapshot`

#### Numpy

In [32]:
st.vars['numpy.float32'][:] = np.ones((2,2,2)) * 3.0
st.vars['numpy.float32'][0] = np.ones((2,2)) * 7.0

In [33]:
print st.vars['numpy.float32'][:]

[[[ 7.  7.]
  [ 7.  7.]]

 [[ 3.  3.]
  [ 3.  3.]]]


#### Obj

You can store objects of a type which you have previously added. For loading you need to make sure that the class (and the store if set manually) is present when you load from the store.

In [34]:
st.vars['obj.nodes'][0,0,0] = Node(1)
st.vars['obj.nodes'][0,1,0] = Node('Second')
st.vars['obj.nodes'][0,0,1] = Node('Third')

In some cases we can also assign one element to a group of elements like it is possible for numbers.

In [35]:
st.vars['obj.nodes'][1] = Node(20)

In [36]:
print st.variables['obj.nodes'][:]
print st.variables['nodes_json'][:]

[[[1 3]
  [2 --]]

 [[4 4]
  [4 4]]]
[u'{"_cls": "Node", "_dict": {"value": 10}}'
 u'{"_cls": "Node", "_dict": {"value": 1}}'
 u'{"_cls": "Node", "_dict": {"value": "Second"}}'
 u'{"_cls": "Node", "_dict": {"value": "Third"}}'
 u'{"_cls": "Node", "_dict": {"value": 20}}']


In [37]:
print st.vars['obj.nodes'][0,0,0]
print type(st.vars['obj.nodes'][0,0,0])

Node(1)
<class '__main__.Node'>


#### lazy

Lazy loading will reconstruct an object using proxies. These proxies behave almost like the loaded object, but will delay loading of the object until it is accessed. Saving for lazy objects is the same as for regular objects. Only loading return a proxy object.

In [38]:
st.vars['lazyobj.nodes'][0,0,0] = Node('First')

The type of the returned object is `LoaderProxy` while the class is the actual class is the baseclass loaded by the store to not trigger loading when the `__class__` attribute is accessed. The actual object can be accessed by `__subject__` and doing so will trigger loading the object. All regular attributes will be delegated to `__subject__.attribute` and also trigger loading.

In [39]:
proxy = st.vars['lazyobj.nodes'][0,0,0]
print 'Type:   ', type(proxy)
print 'Class:  ', proxy.__class__
print 'Content:', proxy.__subject__.__dict__
print 'Access: ', proxy.value

Type:    <class 'openpathsampling.netcdfplus.proxy.LoaderProxy'>
Class:   <class '__main__.Node'>
Content: {'value': 'First'}
Access:  First


### load/save objects

Note that there are now 6 `Node` objects.

In [40]:
print st.nodes[:]

[Node(10), Node(1), Node(Second), Node(Third), Node(20), Node(First)]


In [41]:
obj = Node('BlaBla')
st.nodes.save(obj)
st.save(obj);

In [42]:
print len(st.nodes)

7


Note that we have only 7 objets since both save operations will only store one object.

Get the index of the obj in the storage

In [43]:
print st.idx(obj)

6


And test the different ways to access the contained `json`

#### 1. direct `json` using `variables` in the store

In [44]:
print st.nodes.variables['json'][st.idx(obj)]

{"_cls": "Node", "_dict": {"value": "BlaBla"}}


#### 2. direct `json` using `variables` in the full storage

In [45]:
print st.variables['nodes_json'][st.idx(obj)]

{"_cls": "Node", "_dict": {"value": "BlaBla"}}


#### 3. indirect `json` and reconstruct using `vars` in the store

In [46]:
print st.nodes.vars['json'][st.idx(obj)]
print st.nodes.vars['json'][st.idx(obj)] is obj

Node(BlaBla)
False


#### 4. using the store accessor  `__getitem__` in the store

In [47]:
print st.nodes[st.idx(obj)]
print st.nodes[st.idx(obj)] is obj

Node(BlaBla)
True


One importance difference is that a store like `nodes` has a cache (which we set to 10 before). Using `vars` will not use a store and hence create a new object!